In [2]:
# Import necessary packages
import stim
import numpy as np
import copy
from typing import List, Tuple

In [6]:
class PauliSum:
    class PauliSummand:
        def __init__(self, phase: complex, pauli: stim.PauliString):
            """ 
            Representation for single element of 'PauliSum'

            Params:
            -------
            phase - Phase of the 'PauliSummand' instance
            pauli - Pauli operator of the 'PauliSummand' instance
            """
            self.phase = phase 
            self.pauli = pauli 
            
            # Store basis paulis for each 'PauliSummand' instance
            self.bases = []

        def get_phase(self):
            """ 
            Get phase of PauliSummand

            Returns:
            --------
            self.phase
            """
            return self.phase 

        def set_phase(self, phase: complex):
            """ 
            Set phase of PauliSummand 

            Params:
            -------
            phase - Phase to set to of 'PauliSummand' instance
            """

        def get_pauli(self):
            """ 
            Get pauli of PauliSummand

            Returns:
            --------
            self.pauli
            """
            return self.pauli

        def set_pauli(self, pauli: stim.PauliString):
            """ 
            Set pauli of PauliSummand 

            Params:
            -------
            pauli - Pauli to set to of 'PauliSummand' instance
            """
            self.pauli = pauli

        def get_bases(self):
            """ 
            Get list of basis elements that generate 'PauliSummand' instance

            Returns:
            --------
            self.bases
            """
            return self.bases 

        def add_to_basis(self, basis_ind: int):
            """ 
            Add basis index corresponding to index of basis element in overall list of 
            Pauli bases

            Params:
            -------
            basis_ind - Index of basis element in overall list of Pauli bases
            """
            self.bases.append(basis_ind)

        def __str__(self):
            pauli_str = ""
            pauli_plus_phase = str(self.pauli)
            pauli_plus_phase_len = len(pauli_plus_phase)
            count = pauli_plus_phase_len - 1
            while (pauli_plus_phase[count] == '_' or pauli_plus_phase[count] == 'X' or 
                pauli_plus_phase[count] == 'Y' or pauli_plus_phase[count] == 'Z'):
                pauli_str = pauli_plus_phase[count] + pauli_str 
                count -= 1
            return "[" + str(self.phase) + " " + pauli_str + "]" 

    class PauliSumBasis:
        def __init__(self, *argv):
            """
            Representation for t + 1 (t is number of T-gates) basis elements of 'PauliSum'
            """  
            if len(argv) == 0:
                self.bases = []
                self.num_bases = 0
            else:
                self.bases = argv[0]
                self.num_bases = len(argv[0])

        def get_bases(self):
            """ 
            Get list of bases of 'PauliSum'

            Returns:
            --------
            self.bases
            """
            return self.bases

        def set_bases(self, bases_list: List):
            """ 
            Set basis elements for 'PauliSum'

            Params:
            -------
            bases_list - List of basis elements
            """
            self.bases = bases_list

        def get_num_bases(self):
            """ 
            Returns number of basis elements of 'PauliSum'

            Returns:
            --------
            self.num_bases
            """
            return self.num_bases

        def set_num_bases(self, num_bases: int):
            """ 
            Set number of basis elements of 'PauliSum'

            Params:
            -------
            num_bases - Number of basis elements
            """
            self.num_bases = num_bases

        def incr_num_bases(self):
            """ 
            Increment 'num_bases' variable
            """
            self.num_bases += 1

        def clifford_on_basis(self, cliff: stim.Tableau) -> List:
            """
            Apply clifford operator on all basis_paulis (Change of basis via clifford)

            Params:
            -----------
            cliff - Tableau for clifford operator that is to applied
            """
            self.bases = [cliff(p) for p in self.bases]

        def t_gate_on_basis(self, t_gate_loc: int, num_qubits: int):
            """ 
            Apply T gate on 'self'

            Params:
            -------
            t_gate_loc - Qubit location where T gate is being applied 
            num_qubits - Number of qubits
            """

    def __init__(self, *argv):
        """ 
        Representation of contraction of QCSAT instance
        """
        if len(argv) == 0:
            self.sum = []
            self.bases = PauliSumBasis()
            self.num_bases = 0
        else:
            self.sum = [argv[0]]
            self.bases = PauliSumBasis([argv[0]]) 
            self.num_bases = 1

    def get_sum(self):
        """ 
        Get PauliSum in list representation
        """
        return self.sum 

    def get_bases(self):
        """ 
        Get bases for PauliSum
        """
        return self.bases 

    def set_bases(self, basis_list: List):
        """ 
        Set basis for PauliSum

        Params:
        -------
        basis_list - List of bases for PauliSum
        """
        self.bases = basis_list 

    def get_num_bases(self):
        """ 
        Get number of bases in PauliSum
        """
        return self.num_bases

    def set_num_bases(self, basis_count: int):
        """ 
        Set number of basis elements in PauliSum
        """
        self.num_bases = basis_count

    def __str__(self):
        str_out = ""
        for summand in self.sum:
            str_out += str(summand)
            str_out += " + "
        return str_out[:len(str_out) - 3]

    def apply_tableau(self, t: stim.Tableau):
        """ 
        Apply current stabilizer tableau on PauliSum instance

        Params:
        -------
        t - stabilizer tableau
        """
        for summand in self.sum:
            new_summand = t(summand.get_pauli())
            summand.set_phase(summand.get_phase() * new_summand.sign)
            summand.set_pauli(new_summand)

        
    def combine_like_terms(self):
        op_list = set([elem.get_pauli_str() for elem in self.sum()])

    def add_sum_to_sum(self, p):
        """ 
        Add a PauliSum instance to 'self'

        Params:
        -------
        p - PauliSum instance to be added
        """
        self.sum += p.sum 

    def add_summand(self, s: PauliSummand):
        """ 
        Add a PauliSummand to 'self'

        Params:
        -------
        s - PauliSummand instance to be added 
        """
        self.sum.append(s)

    def apply_t_gate(self, qubit_loc: int = 1):
        """ 
        Simulate application of T gate on PauliSum based on which qubit T gate is 
        being applied to 

        Params:
        -------
        qubit_loc - Location where T gate is being applied
        """
        phase_term = 1/np.sqrt(2)
        added = False 
        new_sum = PauliSum()
        for summand in self.sum:
            if (summand.get_pauli().__str__()[qubit_loc] == '_' or summand.get_pauli().__str__()[qubit_loc] == 'Z'):
                continue
            elif (summand.get_pauli().__str__()[qubit_loc] == 'X'):
                if (added == False):
                    new_summand = PauliSummand(phase_term * summand.get_phase(), stim.PauliString(summand.get_pauli().__str__()[0:qubit_loc] + 'Y' + summand.get_pauli().__str__()[qubit_loc + 1:]))
                    new_sum = PauliSum(new_summand)
                    summand.set_phase(summand.get_phase() * phase_term)
                    added = True
                else:
                    new_summand = PauliSummand(phase_term * summand.get_phase(), stim.PauliString(summand.get_pauli().__str__()[0:qubit_loc] + 'Y' + summand.get_pauli().__str__()[qubit_loc + 1:]))
                    #new_sum_elem = PauliSum(new_summand)
                    summand.set_phase(summand.get_phase() * phase_term)
                    new_sum.add_summand(new_summand)
            else:
                if (added == False):
                    new_summand = PauliSummand(phase_term * summand.get_phase(), stim.PauliString(summand.get_pauli().__str__()[0:qubit_loc] + 'X' + summand.get_pauli().__str__()[qubit_loc + 1:]))
                    new_sum = PauliSum(new_summand)
                    summand.set_phase(-1 * summand.get_phase() * phase_term)

                    added = True
                else:
                    new_summand = PauliSummand(phase_term * summand.get_phase(), stim.PauliString(summand.get_pauli().__str__()[0:qubit_loc] + 'X' + summand.get_pauli().__str__()[qubit_loc + 1:]))
                    #new_sum_elem = PauliSum(new_summand)
                    summand.set_phase(-1 * summand.get_phase() * phase_term)
                    new_sum.add_summand(new_summand)
        return new_sum